In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to loadcd Inputls

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model-input-data/model_input.xlsx


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
tf.__version__

'2.4.1'

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# downloading dataset from tensorflow dataset repo

dataset, info = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', with_info=True)
train_dataset = dataset['train']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incomplete0BMQIX/amazon_us_reviews-train.tfrecord


  0%|          | 0/104975 [00:00<?, ? examples/s]

Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.


In [5]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [6]:
print(train_dataset)
print("\n\n",f"size of dataset : {len(train_dataset)}")

<PrefetchDataset shapes: {data: {customer_id: (), helpful_votes: (), marketplace: (), product_category: (), product_id: (), product_parent: (), product_title: (), review_body: (), review_date: (), review_headline: (), review_id: (), star_rating: (), total_votes: (), verified_purchase: (), vine: ()}}, types: {data: {customer_id: tf.string, helpful_votes: tf.int32, marketplace: tf.string, product_category: tf.string, product_id: tf.string, product_parent: tf.string, product_title: tf.string, review_body: tf.string, review_date: tf.string, review_headline: tf.string, review_id: tf.string, star_rating: tf.int32, total_votes: tf.int32, verified_purchase: tf.int64, vine: tf.int64}}>


 size of dataset : 104975


In [7]:
BUFFER_SIZE = 3000 # decides how many rows of data you want to pre load/pre fetch in the memory for the next epoch.It speeds up the model training process.
# If set to too high,available memory might not be enough.If set to too low,your model might see only data related to one class.So its good to shuffle your 
# data properly before using it for training and setting this value to intermeddiate value considering your dataset size.

BATCH_SIZE = 128 # how many rows of input data you wish to provide in each step of the epoch.So if set to 100 and dataset size is 1000,so there will be 10 steps of size 100 in each epoch.

In [8]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)
len(train_dataset)

104975

In [9]:
for reviews in train_dataset.take(10):
  review_text = reviews['data']
  print(review_text.get('review_body').numpy()) # .numpy() is a tf internal function to convert tensors into numpy arrays.
  print(review_text.get('star_rating'))
  print(tf.where(review_text.get('star_rating')>3,1,0).numpy())

b'Great product.  Great price.  Super fast delivery.  I was pleasantly suprised that there was glue and extra rhinestones incase any of them fall off.  Way better than anywhere else I found, all the way around. Price, product, shipping, GREAT!!!'
tf.Tensor(5, shape=(), dtype=int32)
1
b"I'm thoroughly impressed with this product. Aside of mild curiosity of wanting to get one for some time now, it wasn't until a few days ago when an errant driver nearly (within inches) caused me to total both my, and her, car that the curiosity turned into a need for the &#34;this might be necessary one day&#34; way of thinking. Because after all, there's no &#34;my word against theirs&#34; if you have video in the event of an wreck/insurance case.<br /><br />First thing first, the unit comes with a 16 GB card, which is probably fine for most cases, but because it was on sale, I maxed the device's capacity to 64 GB, which has to be formatted with the BlackVue viewer (a free download from their site). The

In [10]:
## PRE PROCESS THE DATASET

In [11]:
## Tokenizing sentences to words and creating vocab set of unique words. Vocab_size will be usefull in embedding layer implementation.

tokenizer = tfds.features.text.Tokenizer()

vocab_set = set()

for _,review in train_dataset.enumerate():
    review_text = review["data"]
    review_token = tokenizer.tokenize(review_text.get("review_body").numpy())
    vocab_set.update(review_token)
    
vocab_size = len(vocab_set)
vocab_size
    

73738

In [12]:
## Assigning each word in our vocab to an integer value.

encoder = tfds.features.text.TokenTextEncoder(vocab_set,lowercase=True,strip_vocab=True)

In [13]:
encoded_list = set()
for _,reviews in train_dataset.enumerate():
  review_text = reviews['data']
#   print(review_text.get('review_body').numpy())
  encoded_example = encoder.encode(review_text.get('review_body').numpy())
  encoded_list.update(encoded_example)

print(min(encoded_list)) # Notice that it starts from 1 and not 0. Because 0 is used to say no presence of word (used for padding tensors input)
print(max(encoded_list)) # This will be same as the size of our vocab.

3
73738


In [14]:
# To find the word corresponding to the encoded int value.

encoder.decode([10])

'7days'

In [15]:
def encode(text_tensor, label_tensor):
  encoded_text = encoder.encode(text_tensor.numpy())
  label = tf.where(label_tensor>3,1,0)
  return encoded_text, label

In [16]:
def encode_map_fn(tensor):
  
  text = tensor['data'].get('review_body')
  label = tensor['data'].get('star_rating') 

  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int32))

  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label

In [17]:
ar_encoded_data = train_dataset.map(encode_map_fn)

In [18]:
for f0,f1 in ar_encoded_data.take(2):
  print(f0)
  print(f1)
    
# You can see all the tensors output are of different length as the number of words in all sentences were different .

tf.Tensor(
[71627 57849 71627 67848 60444 53146 35515 60068 59840 72761 66222 61553
 69463 59840 72631 68623 56014 58525 57709 56775 41070 64437 45520 49042
 42516 73158 59991 43648 69025 60068 68300 59203 64340 42516 65513 67848
 57849 60045 71627], shape=(39,), dtype=int64)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(
[60068 63139 60823 46258 54689 60916 57849 64928 41070 30169 22902 41070
 65991 67180 49501 31035 72723 67623 21921 64300 60347 63697 26623 57941
 35952 53828 59696 67361 38816 66918 71686 61111 54543 66102 55554 26210
 66640 67180 29448 64389 63244 68623 40443 70492 61553 64340 22902 29185
 65937 35952 56209 72723 64340  6471 60916 52245 73494 66245 31035 58254
  6471 42516 41070 70027 68798 71214 59203 69463 37246 69954  6471 63244
 66176 71924 68409  6471 72512 68391 65669 43268 37492 64340 66473 41070
 66918 28203 49612 53219 48937 48937 32835 69646 32835 64340 65474 31157
 54689 35952 11179 49618 38106 56746 61233 72035 49450 72723 54294 65704
 55721 68798 60347 

In [19]:
TAKE_SIZE=14000

train_data = ar_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE) # Skip the first 14000 records and keep the rest in the training dataset and then shuffle them.
train_data = train_data.padded_batch(BATCH_SIZE) # As the input tensor are of different length,we need to padd them.Here we are saying for every batch of data,
# find the max length of the tensor input and accordingly padd the rest of the data in that batch.The input vector length has to be same for NN input layers.
#  Padding means putting 0 in the place , so that all vectors become of same input.

test_data = ar_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [20]:
vocab_size += 1 # Because of padding we have added 0 also in the tensors,so increasing our vocab size.

In [21]:
next(iter(test_data))

(<tf.Tensor: shape=(128, 779), dtype=int64, numpy=
 array([[71627, 57849, 71627, ...,     0,     0,     0],
        [60068, 63139, 60823, ..., 17609, 73494, 42451],
        [60068, 59840, 71129, ...,     0,     0,     0],
        ...,
        [60068, 65669, 73093, ...,     0,     0,     0],
        [60068, 66186, 65669, ...,     0,     0,     0],
        [72512, 68391, 65669, ...,     0,     0,     0]])>,
 <tf.Tensor: shape=(128,), dtype=int32, numpy=
 array([1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
        1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)>)

In [22]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0],sample_labels[0]

(<tf.Tensor: shape=(779,), dtype=int64, numpy=
 array([71627, 57849, 71627, 67848, 60444, 53146, 35515, 60068, 59840,
        72761, 66222, 61553, 69463, 59840, 72631, 68623, 56014, 58525,
        57709, 56775, 41070, 64437, 45520, 49042, 42516, 73158, 59991,
        43648, 69025, 60068, 68300, 59203, 64340, 42516, 65513, 67848,
        57849, 60045, 71627,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  

In [23]:
for f0,f1 in test_data.take(10):
  print(tf.unique_with_counts(f1)[2].numpy())

[83 45]
[41 87]
[74 54]
[76 52]
[41 87]
[82 46]
[95 33]
[82 46]
[87 41]
[41 87]


In [24]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True)))
# model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)))
model.add(tf.keras.layers.Dense(64,activation="relu"))
model.add(tf.keras.layers.Dropout(64*0.006))
model.add(tf.keras.layers.Dense(1))

In [25]:
import datetime
logdir = os.path.join("/temp/logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/temp/sentiment_analysis.hdf5',verbose=1,save_best_only=True)

model.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=["accuracy"])

In [26]:
# history = model.fit(train_data,epochs=2,validation_data=test_data,callbacks=[tensorboard_callback,checkpointer])
history = model.fit(train_data,epochs=2,validation_data=test_data)

Epoch 1/2
711/711 [==============================] - 336s 432ms/step - loss: 0.6207 - accuracy: 0.6395 - val_loss: 0.5340 - val_accuracy: 0.7655
Epoch 2/2
711/711 [==============================] - 321s 427ms/step - loss: 0.5068 - accuracy: 0.7805 - val_loss: 0.4267 - val_accuracy: 0.8093
